In [47]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import time

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [50]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [51]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [52]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.fc(x)
        return x


In [54]:
model = NeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start = time.time()

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()
    acc = 100 * correct / total
    print(f"Epoch {epoch+1:03d} | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {acc:.2f}% | Time: {time.time()-start:.1f}s")

Epoch 001 | Loss: 1.6883 | Train Acc: 37.75% | Time: 25.1s
Epoch 002 | Loss: 1.2310 | Train Acc: 55.63% | Time: 27.4s
Epoch 003 | Loss: 1.0446 | Train Acc: 63.06% | Time: 25.3s
Epoch 004 | Loss: 0.9013 | Train Acc: 68.59% | Time: 27.4s
Epoch 005 | Loss: 0.7976 | Train Acc: 72.61% | Time: 27.2s
Epoch 006 | Loss: 0.7248 | Train Acc: 75.67% | Time: 25.6s
Epoch 007 | Loss: 0.6554 | Train Acc: 78.05% | Time: 27.7s
Epoch 008 | Loss: 0.6007 | Train Acc: 80.09% | Time: 28.9s
Epoch 009 | Loss: 0.5615 | Train Acc: 81.41% | Time: 30.5s
Epoch 010 | Loss: 0.5192 | Train Acc: 82.88% | Time: 28.9s
Epoch 011 | Loss: 0.4817 | Train Acc: 84.19% | Time: 28.5s
Epoch 012 | Loss: 0.4473 | Train Acc: 85.30% | Time: 28.8s
Epoch 013 | Loss: 0.4285 | Train Acc: 85.70% | Time: 29.3s
Epoch 014 | Loss: 0.4008 | Train Acc: 86.82% | Time: 30.8s
Epoch 015 | Loss: 0.3759 | Train Acc: 87.61% | Time: 28.5s
Epoch 016 | Loss: 0.3629 | Train Acc: 88.05% | Time: 32.6s
Epoch 017 | Loss: 0.3411 | Train Acc: 88.86% | Time: 29.

In [55]:
torch.save(model.state_dict(), 'trained_net.pth')

In [60]:
model = NeuralNet().to(device)
model.load_state_dict(torch.load('trained_net.pth'))

C:\Users\mohib\AppData\Local\Temp\ipykernel_51452\2857926675.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [57]:
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 90.97%


In [59]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = new_transform(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    return image


image_paths = ['images/airplane.jpeg', 'images/dog.jpeg']
images = [load_image(img) for img in image_paths]

model.eval()
with torch.no_grad():
    for image in images:
        output = model(image)
        _, predicted = torch.max(output, 1)
        print(f'predicted: {class_names[predicted.item()]}')


predicted: plane
predicted: dog
